<a href="https://colab.research.google.com/github/Elwing-Chou/ml0716/blob/master/house_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")
train_df

In [ ]:
datas = pd.concat([train_df, test_df], axis=0)
datas = datas.drop(["SalePrice"], axis=1).reset_index(drop=True)
datas

In [ ]:
import numpy as np
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
y = train_df["SalePrice"]
print("skew:", skew(y))
print("lambda:", boxcox_normmax(1 + y))
sns.distplot(y)

In [ ]:
y = train_df["SalePrice"]
y_train_log1p = np.log1p(y)
print("skew:", skew(y_train_log1p))
sns.distplot(y_train_log1p)

In [ ]:
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

In [ ]:
datas_drop = datas.drop(["Id", "PoolQC"], axis=1)

In [ ]:
idx = datas_drop.dtypes != "object"
number_idx = datas_drop.dtypes[idx].index.drop(["MSSubClass"])
number_idx

In [ ]:
import pandas as pd
datas_drop = pd.get_dummies(datas_drop)
datas_drop = pd.get_dummies(datas_drop, columns=["MSSubClass"])
datas_drop

In [ ]:
def getskew(series):
    return skew(series.dropna() + 1)
skewness = (datas_drop[number_idx].apply(getskew)
                  .sort_values(ascending=False))
need_saved = skewness[skewness > 1].index
print(need_saved)
skewness[skewness > 1]

In [ ]:
def save(s):
    lamda = boxcox_normmax(s.dropna() + 1)
    return boxcox1p(s, lamda)
datas_drop[need_saved] = datas_drop[need_saved].apply(save)
skewness = (datas_drop[number_idx].apply(getskew)
                  .sort_values(ascending=False))
skewness

In [ ]:
datas_drop = datas_drop.fillna(datas_drop.median())
datas_drop